In [ ]:
import numpy as np
import pandas as pd

# Importa el notebook que contiene las funciones
%run funciones.ipynb

In [ ]:
# leemos el archivo

ruta_relativa = '../datos/initial-Table.csv'

datos = pd.read_csv(ruta_relativa)

datos.head()

In [ ]:
# Análisis binario  Donde 'AD' es 1, 'Censored'  y 'Other' son 0  

datos['Status'] = datos['Status'].apply(lambda x: 1 if x == 'AD' else 0)

# comprobamos
datos.head()


In [ ]:
# para ser consistente con la nomenclatura de las funciones que se crearon

datos = datos.rename(columns={'Sex': 'sex'})

In [ ]:
cohortes_alemanas = ['ZIM','UHA','DCN','AgeCoDe','FACE']

datos_cohortes_alemanas = preparado_estrategia3(datos, cohortes_alemanas)

display(datos_cohortes_alemanas.sample(3))

In [ ]:
import h5py

# Ruta al archivo HDF5
file_path = '/ceph01/projects/AGRamirez_misc/carpeta_alberto_moreno/datos/transfer_learning/experimento6/genotype.h5'

# Función para imprimir el contenido del archivo HDF5
def print_structure(name, obj):
    print(f"{name}: {obj}")
    if isinstance(obj, h5py.Group):
        print(f"Grupo: {name}")
    elif isinstance(obj, h5py.Dataset):
        print(f"Dataset: {name}, shape: {obj.shape}, dtype: {obj.dtype}")

# Abrir el archivo en modo de solo lectura y listar su contenido
with h5py.File(file_path, 'r') as f:
    print("Estructura completa del archivo HDF5:")
    f.visititems(print_structure)


In [ ]:
import h5py


# Abrir el archivo en modo de solo lectura
with h5py.File(file_path, 'r') as f:
    # Revisar contenido específico del dataset 'data'
    if 'data' in f:
        data = f['data']
        print("\nContenido del dataset 'data':")
        print(f"Shape: {data.shape}")
        print(f"Dtype: {data.dtype}")
        print(f"Attributes: {dict(data.attrs)}")
        
        # Imprimir las primeras 5 filas y 5 columnas del dataset para inspeccionar
        if data.shape[0] > 0:
            print("Data (primeras 5 filas y 5 columnas):")
            print(data[:5, :5])  # Ajusta según la estructura inspeccionada
            print("Data (primeras 5 filas y últimas 5 columnas):")
            print(data[:5, -5:])  # Ajusta según la estructura inspeccionada
            print("Data (primeras 5 filas y columnas 100000 a 100005):")
            print(data[:5, 100000:100005])  # Ajusta según la estructura inspeccionada
        else:
            print("El dataset 'data' está vacío.")
    else:
        print("\nNo se encontró el dataset 'data'.")


In [ ]:
import pandas as pd

# Ruta al archivo SUB_ID.txt
sub_id_path = '/ceph01/projects/AGRamirez_misc/campos/MCI/data/h5files_mixed_SPGER/tmp_files/SUB_ID.txt'

# Cargar el archivo SUB_ID.txt
with open(sub_id_path, 'r') as file:
    sample_ids = file.read().splitlines()

# Crear un DataFrame con los Sample_IDs y sus índices
sample_order_df = pd.DataFrame({
    'Sample_ID': sample_ids,
    'genotype_row': range(len(sample_ids))
})

# Mostrar las primeras filas del DataFrame
print(sample_order_df.head())


In [ ]:
# Crear el diccionario de mapeo
id_to_index = dict(zip(sample_order_df['Sample_ID'], sample_order_df['genotype_row']))

# Verificar el mapeo
print(list(id_to_index.items())[:10])  # Imprime los primeros 10 mapeos para verificar


# Añadir la columna 'genotype_row' usando el mapeo de identificadores
datos_cohortes_alemanas['genotype_row'] = datos_cohortes_alemanas['Sample_ID'].map(id_to_index)

# Mostrar los resultados
print(datos_cohortes_alemanas[['Sample_ID', 'genotype_row']].sample(20))  # Verificar 20 filas



In [ ]:
# comprobamos aquellas muestras que no estaban en genotype
datos_cohortes_alemanas['genotype_row'].isna().sum()

In [ ]:
# Ordenar el DataFrame basado en la columna 'genotype_row'
subject_csv = datos_cohortes_alemanas.sort_values(by='genotype_row')

# Verificar el DataFrame ordenado
display(subject_csv.head())  # Imprime las primeras filas para verificar el orden

In [ ]:
# Filtrar las columnas deseadas, INTRODUCIMOS COVARIABLES, DEBEN SER INTRODUCIDAS COMO cov_1,cov_2,etc
#selected_columns = subject_csv[['Sample_ID', 'Status','genotype_row', 'DataSet']]


# renombramos las columnas:
#subjects = selected_columns.rename(columns={
#    'Sample_ID': 'patient_id',
 #   'Status': 'labels',
 #   'DataSet': 'set',
    
## })


selected_columns = subject_csv[['Sample_ID', 'Status','genotype_row', 'DataSet','Age','sex','apoe2','apoe4','PC1','PC2','PC3','PC4']]

# renombramos las columnas:
subjects = selected_columns.rename(columns={
    'Sample_ID': 'patient_id',
    'Status': 'labels',
    'DataSet': 'set',
    'Age' : 'cov_1',
    'sex' : 'cov_2',
    'apoe2' : 'cov_3',
    'apoe4' : 'cov_4',
    'PC1' : 'cov_5',
    'PC2' : 'cov_6',
    'PC3' : 'cov_7',
    'PC4' : 'cov_8'
    
})

subjects.head()

In [ ]:
# por lo que podemos calcular el weight positive class en los valores de train

# Convertir la columna 'set' a tipo entero
subjects['set'] = subjects['set'].astype(int)

# Convertir la columna 'labels' a tipo entero
subjects['labels'] = subjects['labels'].astype(int)

# Aplicar filtro para quedarnos solo con los valores de set 1
filtered_df = subjects[subjects['set'] == 1]

print(filtered_df.shape)

num_positivos = filtered_df.labels.sum()
num_negativos = len(filtered_df) - num_positivos

print(num_positivos)
print(num_negativos)

In [ ]:
weight_0 = 2385 / (2 * 1760)
weight_1 = 2385 / (2 * 625)

print(weight_0)

print(weight_1)



In [ ]:
# Guardar el archivo subjects con los datos de cohortes_alemanas

import os

# Definir la ruta donde quieres guardar el archivo
ruta_guardado = '/ceph01/projects/AGRamirez_misc/carpeta_alberto_moreno/datos/transfer_learning/experimento5_Spain_Germany/subjects.csv'

# Crear el directorio si no existe
os.makedirs(os.path.dirname(ruta_guardado), exist_ok=True)

# Ahora puedes guardar el DataFrame sin preocuparte de si el directorio existe
subjects.to_csv(ruta_guardado)


In [ ]:
# Añadimos los otrops dos archivos necesarios para que corra GenNet

import shutil

# Rutas de los archivos originales
ruta_topology = '../datos/experimento9_topology_tocha_FACE_cov/topology.csv'
ruta_genotype = '../datos/experimento9_topology_tocha_FACE_cov/genotype.h5'

# Ruta de destino
destino = '/ceph01/projects/AGRamirez_misc/carpeta_alberto_moreno/datos/transfer_learning/experimento5_Spain_Germany/'


# Copiar los archivos
shutil.copy(ruta_topology, destino + 'topology.csv')
shutil.copy(ruta_genotype, destino + 'genotype.h5')
